<a href="https://colab.research.google.com/github/JustMistaken/Design-Briefing-Generator/blob/main/Design_Briefing_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip -q install google-genai

In [45]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
# Instalar Framework ADK de agentes do Google ################################################
!pip install -q google-adk

In [ ]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.5-flash-preview-04-17"

In [ ]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [ ]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [ ]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################

def agente_buscador(area_atuacao, cliente, projeto, data_de_hoje):
    """
    Agente que busca notícias e informações relevantes sobre a área de atuação,
    o cliente e o projeto, para contextualizar o briefing de design.
    """

    buscador = Agent(
        name="agente_buscador",
        model=MODEL_ID,
        description="Agente especializado em buscar informações relevantes para briefings de design.",
        tools=[google_search],
        instruction=f"""
        Você é um assistente de pesquisa especializado em design.
        Sua tarefa é usar a ferramenta de busca do Google (google_search) para coletar informações relevantes
        que ajudarão a contextualizar um briefing de design.

        Você deve buscar informações sobre:
        1.  **Tendências em {area_atuacao}:**
            * Recupere as últimas notícias, artigos e estudos sobre tendências, inovações e melhores práticas em {area_atuacao}.
            * Foque em informações que possam inspirar o processo de design e ajudar a definir a direção criativa.
        2.  **Informações sobre {cliente}:**
            * Busque informações sobre a empresa {cliente}, incluindo sua história, valores, identidade visual, produtos/serviços e posicionamento no mercado.
            * Isso ajudará a garantir que o design esteja alinhado com a identidade da marca.
        3.  **Referências para o projeto {projeto}:**
            * Procure por projetos de design semelhantes a {projeto} que possam servir de inspiração ou referência.
            * Analise o que funciona bem nesses projetos e o que pode ser evitado.

        Atenção:
        * Priorize fontes confiáveis e atualizadas.
        * Seja conciso e objetivo nas suas buscas.
        * Formate a saída de forma clara e organizada, usando títulos e listas quando apropriado.
        """
    )

    entrada_do_agente_buscador = f"""
    Área de atuação: {area_atuacao}
    Cliente: {cliente}
    Projeto: {projeto}
    Data de hoje: {data_de_hoje}
    """

    informacoes_relevantes = call_agent(buscador, entrada_do_agente_buscador)
    return informacoes_relevantes

In [60]:
################################################
# --- Agente 2: Analista de Insights --- #
################################################

def agente_analista(area_atuacao, cliente, projeto, informacoes_relevantes):
    """
    Agente que analisa as informações coletadas e fornece insights relevantes para o briefing.
    """

    analista = Agent(
        name="agente_analista",
        model=MODEL_ID,
        description="Agente especializado em analisar informações e fornecer insights concisos para briefings de design.",
        tools=[],  # Este agente não precisa de ferramentas externas
        instruction=f"""
        Você é um analista de design experiente e perspicaz.
        Sua tarefa é analisar as informações fornecidas sobre a área de atuação, o cliente, o projeto
        e as informações relevantes (tendências, informações sobre o cliente, referências de projeto)
        e fornecer os 5 insights mais acionáveis e relevantes que ajudarão a orientar o briefing e o processo de design.

        Considere os seguintes aspectos em sua análise:

        * **Tendências em {area_atuacao}:** Identifique as tendências mais impactantes para este projeto.
        * **Informações sobre {cliente}:** Destaque os aspectos mais importantes da identidade e contexto do cliente para o design.
        * **Referências para o projeto {projeto}:** Aponte os aprendizados chave de projetos similares.

        Selecione os 5 insights mais cruciais que devem ser considerados para a criação do briefing.
        Apresente cada insight de forma concisa (máximo 2-3 frases por insight), clara e direta, sem formatação especial como asteriscos ou títulos longos.
        Numere cada insight sequencialmente (1., 2., 3., 4., 5.).
        Seja específico sobre como cada insight pode influenciar as perguntas do briefing ou as decisões de design.
        """
    )

    entrada_do_analista = f"""
    Área de atuação: {area_atuacao}
    Cliente: {cliente}
    Projeto: {projeto}
    Informações Relevantes: {informacoes_relevantes}
    """

    analise_e_insights = call_agent(analista, entrada_do_analista)
    return analise_e_insights

In [ ]:
################################################
# --- Agente 3: Gerador de Perguntas --- #
################################################

def agente_gerador_perguntas(area_atuacao, cliente, projeto, informacoes_relevantes):
    """
    Agente que gera perguntas detalhadas para o briefing com o cliente,
    integrando informações relevantes obtidas pelo agente_buscador.
    """

    gerador_perguntas = Agent(
        name="agente_gerador_perguntas",
        model=MODEL_ID,
        description="Agente especializado em gerar perguntas eficazes para briefings de design, considerando o contexto.",
        tools=[],  # Este agente não precisa de ferramentas externas
        instruction=f"""
        Você é um especialista em design e condução de briefings.
        Sua tarefa é gerar uma lista de perguntas detalhadas e eficazes para um briefing com o cliente.

        Use as informações fornecidas sobre a área de atuação, o cliente, o projeto e as informações relevantes
        para gerar perguntas que cubram todos os aspectos relevantes do projeto.

        As Informações Relevantes são muito importantes, use-as para guiar a sua geração de perguntas
        e deixe claro para o usuário que você está utilizando elas.

        Considere os seguintes tópicos ao gerar as perguntas:

        * **Objetivos do Projeto:**
            * Pergunte sobre os objetivos gerais do projeto, os resultados esperados e os indicadores de sucesso.
        * **Público-alvo:**
            * Investigue quem é o público-alvo do projeto, suas necessidades, desejos e preferências.
        * **Mensagem e Tom de Voz:**
            * Explore a mensagem que o projeto deve transmitir e o tom de voz apropriado.
        * **Concorrentes:**
            * Pergunte sobre os principais concorrentes e o que diferencia o projeto deles.
        * **Requisitos Técnicos:**
            * Averigue os requisitos técnicos, restrições e limitações do projeto.
        * **Orçamento e Cronograma:**
            * Discuta o orçamento disponível e o cronograma do projeto.
        * **Informações Relevantes:**
            * Use as informações relevantes (tendências, informações sobre o cliente, etc.) para gerar perguntas mais específicas e aprofundadas.
            * **Destaque como as informações relevantes influenciam as perguntas.**

        Formate a saída como uma lista numerada ou com marcadores.
        Seja claro, conciso e objetivo em suas perguntas.
        """
    )

    entrada_do_gerador_perguntas = f"""
    Área de atuação: {area_atuacao}
    Cliente: {cliente}
    Projeto: {projeto}
    Informações Relevantes: {informacoes_relevantes}
    """

    perguntas_briefing = call_agent(gerador_perguntas, entrada_do_gerador_perguntas)
    return perguntas_briefing

In [55]:
################################################
# --- Agente 4: Apresentador/Revisor --- #
################################################

def agente_apresentador(perguntas_briefing, area_atuacao, cliente, projeto):
    """
    Agente que apresenta as perguntas do briefing ao usuário,
    permite a revisão e formata a lista final.
    """

    apresentador = Agent(
        name="agente_apresentador",
        model=MODEL_ID,
        description="Agente que apresenta, revisa e formata as perguntas do briefing.",
        tools=[],  # Este agente não precisa de ferramentas externas
        instruction=f"""
        Você é um assistente de design experiente em revisar e aprimorar perguntas de briefing.
        Sua tarefa é revisar a lista de perguntas fornecida e apresentar uma versão final e otimizada.

        Considere as melhores práticas para briefings de design ao revisar:
        * Tornar as perguntas mais específicas e claras.
        * Garantir que todas as áreas relevantes sejam cobertas.
        * Usar um tom adequado.
        * Evitar jargões.

        Apresente a lista final de perguntas de forma clara e organizada, seguindo estas diretrizes de formatação para o arquivo de texto final:

        1. Organize as perguntas em tópicos principais, numerando cada tópico sequencialmente (1., 2., 3., etc.).
        2. Inclua um título claro para cada tópico principal.
        3. Liste as sub-perguntas dentro de cada tópico usando marcadores como asteriscos (*) ou hífens (-), com uma indentação para melhor visualização.
        4. Insira uma linha em branco entre cada tópico principal para criar espaçamento.
        5. Remova quaisquer caracteres de formatação de texto como negrito (**) ou outros símbolos desnecessários.

        Exemplo de formatação desejada:

        1. Título do Primeiro Tópico
           * Sub-pergunta 1 do primeiro tópico
           * Sub-pergunta 2 do primeiro tópico

        2. Título do Segundo Tópico
           * Sub-pergunta 1 do segundo tópico
           * Sub-pergunta 2 do segundo tópico

        ... e assim por diante.

        Não inclua explicações sobre as alterações realizadas.
        """
    )

    entrada_do_apresentador = f"""
    Área de atuação: {area_atuacao}
    Cliente: {cliente}
    Projeto: {projeto}
    Perguntas do Briefing: {perguntas_briefing}

    Por favor, revise as perguntas e formate-as conforme as diretrizes fornecidas.
    """

    perguntas_revisadas = call_agent(apresentador, entrada_do_apresentador)
    return perguntas_revisadas

In [61]:
from datetime import date
from google.colab import files
import os

print("🚀 Iniciando o Sistema de Criação de Briefing de Design com 4 Agentes! 🚀\n")

# --- Obter as Informações Iniciais do Usuário ---

data_de_hoje = date.today().strftime("%d/%m/%Y")

# Inserir lógica do sistema de agentes ################################################

area_atuacao = input("Para direcionar a criação do briefing, poderia me informar qual é a sua área de atuação? (Ex: Identidade visual, Design de embalagens, UX/UI, etc.) ")
cliente = input("\nPor favor, insira o nome de projeto para o qual este briefing está sendo criado: ")
projeto = input("\nCerto. Poderia descrever brevemente sobre o que se trata este projeto de design? (Ex: Novo website, Campanha de marketing, Redesign de logo) ")

if not (area_atuacao and projeto):
    print("\nOps! Parece que você esqueceu de preencher alguma informação importante. Por favor, revise e tente novamente.")
else:
    print(f"\nExcelente! Com as informações fornecidas, vamos começar a criar o briefing para o projeto '{projeto}'.\n\n--- 🔍 Pesquisando sobre o tema ---\n")

    informacoes_relevantes = agente_buscador(area_atuacao, cliente, projeto, data_de_hoje)
    print("--- ✅ Analisando as informações ---\n")

    analise_e_insights = agente_analista(area_atuacao, cliente, projeto, informacoes_relevantes)
    print("--- ❓ Gerando questões ---\n")

    perguntas_briefing = agente_gerador_perguntas(area_atuacao, cliente, projeto, analise_e_insights)
    print("--- 💫 Revisando os dados ---\n")

    perguntas_revisadas = agente_apresentador(perguntas_briefing, area_atuacao, cliente, projeto)
    print("--- 💛 Resultado ---\n")

    # Cria o conteúdo para o arquivo TXT (título + insights + perguntas revisadas)
    nome_arquivo = f"briefing_design_{cliente.replace(' ', '_')}_{projeto.replace(' ', '_')}_{data_de_hoje.replace('/', '_')}.txt"
    titulo_briefing = f"Briefing: {projeto} para {cliente} \n\n"  # Cria o título com o nome do projeto
    conteudo_arquivo = titulo_briefing + "-- Insights Gerados --\n\n" + analise_e_insights + "\n\n-- Perguntas do Briefing --\n\n" + perguntas_revisadas  # Concatena título, insights e perguntas

    # Salva o conteúdo no arquivo local
    with open(nome_arquivo, "w") as f:
        f.write(conteudo_arquivo)

    # Disponibiliza o arquivo para download
    files.download(nome_arquivo)

    print(f"\n🎉 O briefing com as perguntas revisadas foi gerado e o arquivo '{nome_arquivo}' está pronto para download.")

🚀 Iniciando o Sistema de Criação de Briefing de Design com 4 Agentes! 🚀
Para direcionar a criação do briefing, poderia me informar qual é a sua área de atuação? (Ex: Identidade visual, Design de embalagens, UX/UI, etc.) UX/UI

Por favor, insira o nome de projeto para o qual este briefing está sendo criado: Liga de Favela

Certo. Poderia descrever brevemente sobre o que se trata este projeto de design? (Ex: Novo website, Campanha de marketing, Redesign de logo) Novo aplicativo mobile

Excelente! Com as informações fornecidas, vamos começar a criar o briefing para o projeto 'Novo aplicativo mobile'.

--- 🔍 Pesquisando sobre o tema ---

--- ✅ Analisando as informações ---

--- ❓ Gerando questões ---

--- 💫 Revisando os dados ---

--- 💛 Resultado ---



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


🎉 O briefing com as perguntas revisadas foi gerado e o arquivo 'briefing_design_Liga_de_Favela_Novo_aplicativo_mobile_18_05_2025.txt' está pronto para download.
